In [136]:
import datetime
import math
import os
import time
import pandas as pd
BASE_DIR = os.path.abspath('')


In [2]:
"""
A function to convert standard latitude
and longitude coordinates to a mercator
coordinate system
"""
def mercator(lat, lon):
    r_major = 6378137.000
    merc_lon = r_major * math.radians(lon)
    try:
        scale = merc_lon/lon
    except ZeroDivisionError:
        return None
    merc_lat = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + lat * (math.pi/180.0)/2.0)) * scale

    return (merc_lat, merc_lon)


In [3]:
map_dimension = {
    'portland_big': {
        'lat_range': (-13665127.619454736, -13644858.232612623),
        'lon_range': (5696480.836269927, 5710927.521886856)
    }
}


In [4]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models.annotations import Title
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.io import export_png

"""
A function to plot mercator coordinates over
Portland map. Alter point alpha and color
to your desire.
"""
def plot_bokeh(x_list, y_list, count): 
    # Set map boundaries (mercator) and axis type
    fig_map = map_dimension['portland_big']
    p = figure(x_range=fig_map['lat_range'],
               y_range=fig_map['lon_range'],
               x_axis_type="mercator", y_axis_type="mercator")
    
    # Tile Source for CartoDB Tile Service
    p.add_tile(CARTODBPOSITRON)
    # Set styling for Metro bus coordinate   
#     p.square(x=y_list[1],
#              y=x_list[1],
#              color='blue',
#              fill_alpha=0.01)
    
    p.triangle(x=y_list,
             y=x_list,
             color='blue',
             size=5,
             fill_alpha=.1)

    
    # Set chart title - date + time
    title = Title()
    title.text = f"AC Transit Buses - {datetime.datetime.now().strftime('%m/%d/%Y  %H:%M')}"
    p.title = title
    
    # For displaying map in notebook - comment out to avoid cluttering
    output_notebook()
    show(p)
    
    # Export as .png
#     export_png(p, filename="plot{}.png".format(count))
    

In [5]:
plot_bokeh(1)

TypeError: plot_bokeh() missing 2 required positional arguments: 'y_list' and 'count'

In [188]:
csv_file=pd.read_csv("triMetData.csv")

In [189]:
a,b=mercator(float(csv_file.latitude[0]),float(csv_file.longitude[0]))

In [190]:
x=(mercator(float(i),float(j)) for i,j in (csv_file.latitude, csv_file.longitude))

In [191]:
plot_bokeh([a],[b],1)

Loading BokehJS ...

In [37]:
min([float(i) for i in list(csv_file.delay) if i is not 'delay'])

ValueError: could not convert string to float: 'delay'

In [192]:
# get colormap scale of delay

delay=[float(i) for i in list(csv_file.delay.unique()) if str(i).isnumeric() or str(i)[1:].isnumeric()]
five_percent=.05*len(delay)
delay.sort()
trimmed_delay=delay[int(five_percent):len(delay)-int(five_percent)]
print(len(delay),len(trimmed_delay))
# plt.hist(trimmed_delay)
# plt.show()

delay_min=min(trimmed_delay)
delay_max=max(trimmed_delay)

1013 913


In [193]:
# get a colormap from matplotlib
colormap =cm.get_cmap("gist_rainbow") #choose any matplotlib colormap here

# define maximum and minimum for cmap
colorspan=[delay_min,delay_max]

# create a color channel with a value between 0 and 1
# outside the colorspan the value becomes 0 (left) and 1 (right)
cmap_input=np.interp(np.sqrt(x*x+y*y),colorspan,[0,1],left=0,right=1)

# use colormap to generate rgb-values
# second value is alfa (not used)
# third parameter gives int if True, otherwise float
A_color=colormap(cmap_input,1,True)

# convert to hex to fit to bokeh
bokeh_colors = ["#%02x%02x%02x" % (r, g, b) for r, g, b in A_color[:,0:3]]

TypeError: unsupported operand type(s) for *: 'generator' and 'generator'

In [73]:
uniq=list(csv_file.datetime.unique())

In [74]:
first=csv_file[csv_file.datetime.isin([uniq[5]])]

In [75]:
_merc=[]
for i,row in first.iterrows():
    _merc.append(mercator(float(row['latitude']),float(row['longitude'])))

In [87]:
x=[i[0] for i in _merc]
y=[i[1] for i in _merc]

In [187]:
import numpy as np

from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin
n = 50000
# x = np.random.standard_normal(n)
# y = np.random.standard_normal(n)
# x=[i for i in a]
# y=[i for i in b]

bins = hexbin(-np.array(x), np.array(y), 0.85)
# print(bins)

p = figure(tools="", match_aspect=True, background_fill_color='#440154')
p.grid.visible = False

p.hex_tile(q="q", r="r", size=0.1, line_color=None, source=bins,
           fill_color=linear_cmap('counts', 'Viridis256', 0, max(bins.counts)))


# export_png(p, filename="plot{}.png".format(1))
show(p)

In [77]:
plot_bokeh(x,y,1)

Loading BokehJS ...

In [18]:
try:
    os.mkdir('data')
    os.chdir('data')
except FileExistsError:
    pass

In [7]:
for index, val in enumerate(uniq):
    item=csv_file[csv_file.datetime.isin([val])]
    _merc=[]
    try:
        for i,row in item.iterrows():
            _merc.append(mercator(float(row['latitude']),float(row['longitude'])))
    except ValueError:
        continue
    x=[i[0] for i in _merc]
    y=[i[1] for i in _merc]
    if index==1:
        break
    plot_bokeh(x,y,index)

NameError: name 'uniq' is not defined

In [34]:
# parallel computing
import concurrent.futures


def map_threads(func, _iterable):
    """Map function with iterable object in using thread pools."""
    with concurrent.futures.ThreadPoolExecutor() as executor:
        result = executor.map(func, _iterable)
    return result


def map_processes(func, _iterable):
    """Map function with iterable object in using process pools."""
    with concurrent.futures.ProcessPoolExecutor() as executor:
        result = executor.map(func, _iterable)
    return result

In [152]:
# parallel computing
x=[]
y=[]
z=[i for i in uniq]
for index, val in enumerate(uniq):
    item=csv_file[csv_file.datetime.isin([val])]
    _merc=[]
    try:
        for i,row in item.iterrows():
            _merc.append(mercator(float(row['latitude']),float(row['longitude'])))
    except ValueError:
        continue
    x.append([i[0] for i in _merc])
    y.append([i[1] for i in _merc])
    x.append(row['latitude'])
    y.append(row['longitude'])
    

    
args=zip(x,y,z)
print(args)

In [180]:
x=list(csv_file.latitude)
y=list(csv_file.longitude)

In [92]:
import copy
_args=copy.deepcopy(args)
a,b,c=zip(*_args)

In [25]:
list(zip(range(1),range(2)))

[(0, 0)]

In [194]:
# x=[]
# y=[]
# [x.extend(i) for i in a]
# [y.extend(i) for i in b]
x=[float(i)*100 for i in x if i !='latitude']
y=[float(i)*100 for i in y if i!='longitude']
x,y

ValueError: too many values to unpack (expected 2)

In [31]:
%%bash
mv *.png ./data

In [89]:
import copy
_args=copy.deepcopy(args)

NameError: name 'args' is not defined

In [42]:
def parallel_plot_bokeh(args): 
    # Set map boundaries (mercator) and axis type
    x_list, y_list, count = args
    fig_map = map_dimension['portland_big']
    p = figure(x_range=fig_map['lat_range'],
               y_range=fig_map['lon_range'],
               x_axis_type="mercator", y_axis_type="mercator")
    
    # Tile Source for CartoDB Tile Service
    p.add_tile(CARTODBPOSITRON)
    # Set styling for Metro bus coordinate   
#     p.square(x=y_list[1],
#              y=x_list[1],
#              color='blue',
#              fill_alpha=0.01)
    
    p.circle(x=y_list,
             y=x_list,
             color='blue',
             size=5,
             fill_alpha=.1)

    
    # Set chart title - date + time
    title = Title()
    title.text = f"AC Transit Buses - {datetime.datetime.now().strftime('%m/%d/%Y  %H:%M')}"
    p.title = title
    
#     # For displaying map in notebook - comment out to avoid cluttering
#     output_notebook()
#     show(p)
    
    # Export as .png
    export_png(p, filename="plot{}.png".format(count))
    

In [44]:
map_processes(parallel_plot_bokeh,args)

KeyboardInterrupt: 

In [43]:
args=copy.deepcopy(_args)

       q   r  counts
0    -27  10       1
1    -27  16       1
2    -25  12       1
3    -25  19       1
4    -24  -5       1
...   ..  ..     ...
1431  24 -10       1
1432  24  -9       1
1433  24  -8       2
1434  25 -22       1
1435  25  -8       1

[1436 rows x 3 columns]


In [62]:
import numpy as np
import matplotlib.cm as cm

import bokeh.plotting as bk

# generate data
N = 4000
x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
radii = np.random.random(size=N) * 1.5

# get a colormap from matplotlib
colormap =cm.get_cmap("gist_rainbow") #choose any matplotlib colormap here

# define maximum and minimum for cmap
colorspan=[40,140]

# create a color channel with a value between 0 and 1
# outside the colorspan the value becomes 0 (left) and 1 (right)
cmap_input=np.interp(np.sqrt(x*x+y*y),colorspan,[0,1],left=0,right=1)
print(cmap_input)

# use colormap to generate rgb-values
# second value is alfa (not used)
# third parameter gives int if True, otherwise float
A_color=colormap(cmap_input,1,True)

# convert to hex to fit to bokeh
bokeh_colors = ["#%02x%02x%02x" % (r, g, b) for r, g, b in A_color[:,0:3]]

# create the plot-
p = bk.figure(title="Example of importing colormap from matplotlib")

p.scatter(x, y, radius=radii,
          fill_color=bokeh_colors, fill_alpha=0.6,
          line_color=None)

bk.output_file("rainbow.html")

bk.show(p)  # open a browser

[0.33214594 0.48407327 0.06071535 ... 0.20169803 0.60944246 0.4149864 ]
